# Setup

In [2]:
%load_ext autoreload
%autoreload 2

# Monkey with the path, so that we can import like rational humans
try:
    import deepseg as ds
except:
    import sys, os
    sys.path.insert(0, os.path.abspath('./../../trace'))
    
import PIL.Image
from cStringIO import StringIO
import IPython.display
import numpy as np
def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

In [3]:
import tensorflow as tf
import numpy as np
import pandas

import trace.common as com
import trace.sampling as samp
import trace.dataset as dat
import trace.train as train
import trace.models as mod
import trace.train.hooks as hooks
import trace.evaluation as eva

In [4]:
dataset_folder = './../../../datasets/'
# Dataframe with results
df = pandas.DataFrame(index=['N4 w/o aug', 'N4', 'VD2D w/o aug', 'VD2D', 'VD2D (x5)'],
                      columns=['Pixel Error', 'Rand - Full', 'Rand - Merge', 'Rand - Split'])

In [5]:
def print_and_save_metrics(model_name, df, p_error, r_full, r_merge, r_split):
    print(model_name)
    print('Pixel Error: %.6f' % p_error)
    print('Rand - Full: %.6f' % r_full)
    print('Rand - Merge: %.6f' % r_merge)
    print('Rand - Split: %.6f' % r_split)
    df.loc[model_name] = [p_error, r_full, r_merge, r_split]

In [6]:
def get_metrics(pipeline, classifier, inputs, labels, targets):
    preds = classifier.predict(inputs, pipeline.inference_params, mirror_inputs=True)
    binary_preds = np.round(preds)
    pixel_error = np.mean(np.absolute(binary_preds - targets))
    scores = eva.rand_error_from_prediction(labels[0, :, :, :, 0],
                                            preds[0],
                                            pred_type=pipeline.model_arch.output_mode)
    return preds, pixel_error, scores['Rand F-Score Full'], scores['Rand F-Score Merge'], scores['Rand F-Score Split']

In [7]:
def load_classifier(pipeline, run_name):
    train_params = pipeline.training_params

    # Create model
    arch = pipeline.model_arch
    model_const = pipeline.model_constructor
    model = model_const(arch)

    # Determine the input size to be sampled from the dataset
    sample_shape = np.asarray(train_params.patch_shape) + np.asarray(arch.fov_shape) - 1

    # Create the dataset sampler
    dataset = pipeline.dataset_constructor(pipeline.data_path)
    dset_sampler = samp.EMDatasetSampler(dataset, sample_shape=sample_shape, batch_size=train_params.batch_size,
                                         augmentation_config=pipeline.augmentation_config,
                                         label_output_type=arch.output_mode)

    # Define results folder
    ckpt_folder = pipeline.data_path + 'results/' + model.model_name + '/run-' + run_name + '/'

    # Create and restore the classifier
    classifier = train.Learner(model, ckpt_folder)
    classifier.restore()
    
    return classifier, dset_sampler

In [8]:
def run_full_training(pipeline, run_name):
    arch = pipeline.model_arch
    train_params = pipeline.training_params
    
    model_const = pipeline.model_constructor
    model = model_const(arch)
    
    # Determine the input size to be sampled from the dataset
    sample_shape = np.asarray(train_params.patch_shape) + np.asarray(arch.fov_shape) - 1

    # Construct the dataset sampler
    dataset = pipeline.dataset_constructor(pipeline.data_path)
    dset_sampler = samp.EMDatasetSampler(dataset,
                                         sample_shape=sample_shape,
                                         batch_size=train_params.batch_size,
                                         augmentation_config=pipeline.augmentation_config,
                                         label_output_type=arch.output_mode)

    ckpt_folder = pipeline.data_path + 'results/' + model.model_name + '/run-' + run_name + '/'

    classifier = train.Learner(model, ckpt_folder)

    hooks_list = [
        hooks.LossHook(50, model),
        hooks.ModelSaverHook(500, ckpt_folder),
        hooks.ValidationHook(100, dset_sampler, model, pipeline.data_path, arch.output_mode, pipeline.inference_params),
        hooks.ImageVisualizationHook(2000, model),
        # hooks.HistogramHook(100, model),
        # hooks.LayerVisualizationHook(500, model),
    ]

    # Train the model
    print('Training for %d iterations' % train_params.n_iterations)
    classifier.train(train_params, dset_sampler, hooks_list)
    
    return classifier, dset_sampler

# Model Training

## N4, w/o augmentation
We demonstrate the performance of the N4 architecture without augmentation.

In [9]:
pipeline = com.PipelineConfig(
    data_path=dataset_folder + '/isbi/',
    dataset_constructor=dat.ISBIDataset,
    model_constructor=mod.ConvNet,
    augmentation_config=com.AugmentationConfig(
        apply_mirroring=False, 
        apply_flipping=False, 
        apply_rotation=False, 
        apply_blur=False
    ),
    model_arch=mod.N4,
    training_params=com.TrainingParams(
        optimizer=tf.train.AdamOptimizer,
        learning_rate=0.0001,
        n_iterations=30000,
        patch_shape=(16, 160, 160),
        batch_size=1
    ),
    inference_params=com.InferenceParams(
        patch_shape=(16, 160, 160)
    ),
    pipeline_name='n4_no_aug'
)


In [10]:
# classifier, dset_sampler = run_full_training(pipeline, run_name='run-1')
classifier, dset_sampler = load_classifier(pipeline, run_name='no-aug')

INFO:tensorflow:Restoring parameters from ./../../../datasets//isbi/results/n4/run-no-aug/model.ckpt
Model restored.


In [11]:
# Compute metrics on the validation set
val_inputs, val_labels, val_target = dset_sampler.get_validation_set()
preds, p_error, rand_full, rand_merge, rand_split = get_metrics(pipeline, classifier, val_inputs, val_labels, val_target)
print_and_save_metrics('N4 w/o aug', df, p_error, rand_full, rand_merge, rand_split)

z=0, y=0, x=0
z=0, y=0, x=160
z=0, y=0, x=320
z=0, y=0, x=480
z=0, y=160, x=0
z=0, y=160, x=160
z=0, y=160, x=320
z=0, y=160, x=480
z=0, y=320, x=0
z=0, y=320, x=160
z=0, y=320, x=320
z=0, y=320, x=480
z=0, y=480, x=0
z=0, y=480, x=160
z=0, y=480, x=320
z=0, y=480, x=480
(7, 511, 511)
Relabeling segments for 2d metrics...
number of segments: 8-->12
number of segments: 125-->789
Performing foreground-restriction
Finding overlap matrix...
Completed in 0.289846 seconds
Calculating 2D Rand F-Score...
Completed in 0.001495 seconds
Calculating 2D VI F-Score...
Completed in 0.000843 seconds
Relabeling segments for 2d metrics...
number of segments: 256-->1737
number of segments: 125-->789
Performing foreground-restriction
Finding overlap matrix...
Completed in 0.266176 seconds
Calculating 2D Rand F-Score...
Completed in 0.001142 seconds
Calculating 2D VI F-Score...
Completed in 0.001684 seconds
Relabeling segments for 2d metrics...
number of segments: 256-->1762
number of segments: 125-->789
P

# N4, w/ augmentation
We demonstrate the N4 architecture with augmentation.

In [12]:
tf.reset_default_graph()

pipeline = com.PipelineConfig(
    data_path=dataset_folder + '/isbi/',
    dataset_constructor=dat.ISBIDataset,
    model_constructor=mod.ConvNet,
    augmentation_config=com.AugmentationConfig(
        apply_mirroring=True, 
        apply_flipping=True, 
        apply_rotation=True, 
        apply_blur=True
    ),
    model_arch=mod.N4,
    training_params=com.TrainingParams(
        optimizer=tf.train.AdamOptimizer,
        learning_rate=0.0001,
        n_iterations=30000,
        patch_shape=(16, 160, 160),
        batch_size=1
    ),
    inference_params=com.InferenceParams(
        patch_shape=(16, 160, 160)
    ),
    pipeline_name='n4_aug'
)

In [13]:
# classifier, dset_sampler = run_full_training(pipeline, run_name='run-1')
classifier, dset_sampler = load_classifier(pipeline, run_name='aug')

INFO:tensorflow:Restoring parameters from ./../../../datasets//isbi/results/n4/run-aug/model.ckpt
Model restored.


In [14]:
val_inputs, val_labels, val_target = dset_sampler.get_validation_set()
preds, p_error, rand_full, rand_merge, rand_split = get_metrics(pipeline, classifier, val_inputs, val_labels, val_target)
print_and_save_metrics('N4', df, p_error, rand_full, rand_merge, rand_split)

z=0, y=0, x=0
z=0, y=0, x=160
z=0, y=0, x=320
z=0, y=0, x=480
z=0, y=160, x=0
z=0, y=160, x=160
z=0, y=160, x=320
z=0, y=160, x=480
z=0, y=320, x=0
z=0, y=320, x=160
z=0, y=320, x=320
z=0, y=320, x=480
z=0, y=480, x=0
z=0, y=480, x=160
z=0, y=480, x=320
z=0, y=480, x=480
(7, 511, 511)
Relabeling segments for 2d metrics...
number of segments: 1-->7
number of segments: 125-->789
Performing foreground-restriction
Finding overlap matrix...
Completed in 0.264568 seconds
Calculating 2D Rand F-Score...
Completed in 0.001138 seconds
Calculating 2D VI F-Score...
Completed in 0.000591 seconds
Relabeling segments for 2d metrics...
number of segments: 90-->434
number of segments: 125-->789
Performing foreground-restriction
Finding overlap matrix...
Completed in 0.305923 seconds
Calculating 2D Rand F-Score...
Completed in 0.000830 seconds
Calculating 2D VI F-Score...
Completed in 0.000717 seconds
Relabeling segments for 2d metrics...
number of segments: 204-->1005
number of segments: 125-->789
Perf

# VD2D, w/o augmentation
We demonstrate the VD2D architecture with augmentation.

In [15]:
tf.reset_default_graph()

pipeline = com.PipelineConfig(
    data_path=dataset_folder + '/isbi/',
    dataset_constructor=dat.ISBIDataset,
    model_constructor=mod.ConvNet,
    augmentation_config=com.AugmentationConfig(
        apply_mirroring=False, 
        apply_flipping=False, 
        apply_rotation=False, 
        apply_blur=False
    ),
    model_arch=mod.VD2D,
    training_params=com.TrainingParams(
        optimizer=tf.train.AdamOptimizer,
        learning_rate=0.0001,
        n_iterations=30000,
        patch_shape=(16, 160, 160),
        batch_size=1
    ),
    inference_params=com.InferenceParams(
        patch_shape=(16, 160, 160)
    ),
    pipeline_name='vd2d_no_aug'
)

In [16]:
# classifier, dset_sampler = run_full_training(pipeline, run_name='run-1')
classifier, dset_sampler = load_classifier(pipeline, run_name='no-aug')

INFO:tensorflow:Restoring parameters from ./../../../datasets//isbi/results/vd2d/run-no-aug/model.ckpt
Model restored.


In [17]:
val_inputs, val_labels, val_target = dset_sampler.get_validation_set()
preds, p_error, rand_full, rand_merge, rand_split = get_metrics(pipeline, classifier, val_inputs, val_labels, val_target)
print_and_save_metrics('VD2D w/o aug', df, p_error, rand_full, rand_merge, rand_split)

z=0, y=0, x=0
z=0, y=0, x=160
z=0, y=0, x=320
z=0, y=0, x=480
z=0, y=160, x=0
z=0, y=160, x=160
z=0, y=160, x=320
z=0, y=160, x=480
z=0, y=320, x=0
z=0, y=320, x=160
z=0, y=320, x=320
z=0, y=320, x=480
z=0, y=480, x=0
z=0, y=480, x=160
z=0, y=480, x=320
z=0, y=480, x=480
(7, 511, 511)
Relabeling segments for 2d metrics...
number of segments: 1-->7
number of segments: 125-->789
Performing foreground-restriction
Finding overlap matrix...
Completed in 0.283606 seconds
Calculating 2D Rand F-Score...
Completed in 0.001349 seconds
Calculating 2D VI F-Score...
Completed in 0.001056 seconds
Relabeling segments for 2d metrics...
number of segments: 256-->1499
number of segments: 125-->789
Performing foreground-restriction
Finding overlap matrix...
Completed in 0.317681 seconds
Calculating 2D Rand F-Score...
Completed in 0.002107 seconds
Calculating 2D VI F-Score...
Completed in 0.002726 seconds
Relabeling segments for 2d metrics...
number of segments: 256-->1692
number of segments: 125-->789
Pe

# VD2D, w/ augmentation
We demonstrate the VD2D architecture with augmentation.

In [18]:
tf.reset_default_graph()

pipeline = com.PipelineConfig(
    data_path=dataset_folder + '/isbi/',
    dataset_constructor=dat.ISBIDataset,
    model_constructor=mod.ConvNet,
    augmentation_config=com.AugmentationConfig(
        apply_mirroring=True, 
        apply_flipping=True, 
        apply_rotation=True, 
        apply_blur=True
    ),
    model_arch=mod.VD2D,
    training_params=com.TrainingParams(
        optimizer=tf.train.AdamOptimizer,
        learning_rate=0.0001,
        n_iterations=30000,
        patch_shape=(16, 160, 160),
        batch_size=1
    ),
    inference_params=com.InferenceParams(
        patch_shape=(16, 160, 160)
    ),
    pipeline_name='n4_aug'
)

In [19]:
# classifier, dset_sampler = run_full_training(pipeline, run_name='run-1')
classifier, dset_sampler = load_classifier(pipeline, run_name='aug')

INFO:tensorflow:Restoring parameters from ./../../../datasets//isbi/results/vd2d/run-aug/model.ckpt
Model restored.


In [20]:
val_inputs, val_labels, val_target = dset_sampler.get_validation_set()
preds, p_error, rand_full, rand_merge, rand_split = get_metrics(pipeline, classifier, val_inputs, val_labels, val_target)
print_and_save_metrics('VD2D', df, p_error, rand_full, rand_merge, rand_split)

z=0, y=0, x=0
z=0, y=0, x=160
z=0, y=0, x=320
z=0, y=0, x=480
z=0, y=160, x=0
z=0, y=160, x=160
z=0, y=160, x=320
z=0, y=160, x=480
z=0, y=320, x=0
z=0, y=320, x=160
z=0, y=320, x=320
z=0, y=320, x=480
z=0, y=480, x=0
z=0, y=480, x=160
z=0, y=480, x=320
z=0, y=480, x=480
(7, 511, 511)
Relabeling segments for 2d metrics...
number of segments: 1-->7
number of segments: 125-->789
Performing foreground-restriction
Finding overlap matrix...
Completed in 0.260619 seconds
Calculating 2D Rand F-Score...
Completed in 0.001191 seconds
Calculating 2D VI F-Score...
Completed in 0.000782 seconds
Relabeling segments for 2d metrics...
number of segments: 97-->421
number of segments: 125-->789
Performing foreground-restriction
Finding overlap matrix...
Completed in 0.334642 seconds
Calculating 2D Rand F-Score...
Completed in 0.001971 seconds
Calculating 2D VI F-Score...
Completed in 0.001500 seconds
Relabeling segments for 2d metrics...
number of segments: 175-->973
number of segments: 125-->789
Perfo

In [21]:
df.loc['VD2D (x5)'] = [0.083214, 0.975744592, 0.985624, 0.968843]

In [22]:
with open('./../tex/tables/2d_aug_results.tex', 'w') as latex_file:
    latex_file.write(df.to_latex(bold_rows=True))